In [1]:
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


# Select your features (columns)

In [4]:
# Set X equal to the entire data set, except for the first column
X = df.iloc[:,1:]

# Set y equal to the first column
y = df.iloc[:,0]

In [5]:
# Search for top 10 features according to feature importances
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=250,learning_rate=1.0,max_depth=1,random_state=0)
model.fit(X,y)
model.feature_importances_

array([2.60688858e-01, 2.46980546e-01, 1.75985959e-01, 2.03134802e-02,
       2.32487160e-03, 4.59675115e-04, 1.85726096e-04, 5.00055556e-03,
       4.10157937e-04, 2.72422069e-04, 4.31920796e-03, 9.41308828e-04,
       2.38202743e-03, 8.88944750e-03, 3.83105489e-02, 2.31265322e-04,
       2.50125549e-03, 1.67951790e-03, 4.52128956e-04, 2.29894003e-02,
       9.63857808e-04, 5.42085677e-03, 1.26294956e-04, 2.86544032e-04,
       7.19421013e-04, 4.61592239e-04, 7.96938647e-02, 2.74682994e-03,
       9.71028723e-04, 9.81578136e-02, 1.08029809e-03, 8.15077613e-04,
       3.65355791e-04, 1.71499278e-03, 1.93201650e-03, 1.31314682e-03,
       9.42795086e-04, 2.23334203e-03, 1.79633572e-03, 2.94017676e-03])

In [6]:
# Store the top 10 features as a series, using the column headers as the index
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_nt        0.260689
koi_fpflag_ss        0.246981
koi_fpflag_co        0.175986
koi_steff_err1       0.098158
koi_model_snr        0.079694
koi_duration_err1    0.038311
koi_prad             0.022989
koi_fpflag_ec        0.020313
koi_duration         0.008889
koi_prad_err2        0.005421
dtype: float64

In [7]:
# Set features based on feature importances
X = df[feat_imp.index]

# Use koi_disposition for y values
y = df['koi_disposition']

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_steff_err1,koi_model_snr,koi_duration_err1,koi_prad,koi_fpflag_ec,koi_duration,koi_prad_err2
1293,0,0,0,84,38.2,0.0331,1.65,0,1.1267,-0.15
4292,0,0,0,61,18.0,0.0769,1.33,0,0.9490,-0.07
5385,0,1,0,144,151.8,0.0432,22.89,0,5.1822,-1.49
6754,1,0,0,193,13.7,1.2000,3.64,0,20.5200,-0.38
6125,0,0,0,182,12.1,0.2990,0.75,1,2.5320,-0.22


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler object
X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [11]:
from sklearn.linear_model import LogisticRegression

# Initialize model
model = LogisticRegression()

# Train the model
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [12]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8239557505245089
Testing Data Score: 0.8323798627002288


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
predictions = model.predict(X_test_scaled)
df_pred = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
df_pred.head()

,Actual,Predicted
1903,FALSE POSITIVE,FALSE POSITIVE
1574,CONFIRMED,CONFIRMED
708,CONFIRMED,CONFIRMED
1406,CONFIRMED,CONFIRMED
1062,CANDIDATE,CONFIRMED


In [14]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
import numpy as np

# Create the GridSearchCV model
#param_grid = {'C':np.logspace(-4, 4, 20),'penalty':['l1','l2']}

param_grid = [  
  {'C':np.logspace(-3,3,7), 'penalty':['l1','l2']},
 ]
grid = GridSearchCV(model, param_grid, cv=10, verbose=3)

In [15]:

# Train the model with GridSearch
best_model = grid.fit(X_train_scaled, y_train)

Fitting 10 folds for each of 14 candidates, totalling 140 fits
[CV 1/10] END ................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 2/10] END ................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 3/10] END ................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 4/10] END ................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 5/10] END ................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 6/10] END ................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 7/10] END ................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 8/10] END ................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 9/10] END ................C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 10/10] END ...............C=0.001, penalty=l1;, score=nan total time=   0.0s
[CV 1/10] END ..............C=0.001, penalty=l2;, score=0.501 total time=   0.0s
[CV 2/10] END ..............C=0.001, penalty=l

[CV 1/10] END ..............C=100.0, penalty=l2;, score=0.857 total time=   0.1s
[CV 2/10] END ..............C=100.0, penalty=l2;, score=0.890 total time=   0.0s
[CV 3/10] END ..............C=100.0, penalty=l2;, score=0.882 total time=   0.0s
[CV 4/10] END ..............C=100.0, penalty=l2;, score=0.885 total time=   0.1s
[CV 5/10] END ..............C=100.0, penalty=l2;, score=0.868 total time=   0.0s
[CV 6/10] END ..............C=100.0, penalty=l2;, score=0.876 total time=   0.0s
[CV 7/10] END ..............C=100.0, penalty=l2;, score=0.857 total time=   0.1s
[CV 8/10] END ..............C=100.0, penalty=l2;, score=0.872 total time=   0.1s
[CV 9/10] END ..............C=100.0, penalty=l2;, score=0.865 total time=   0.0s
[CV 10/10] END .............C=100.0, penalty=l2;, score=0.882 total time=   0.0s
[CV 1/10] END ...............C=1000.0, penalty=l1;, score=nan total time=   0.0s
[CV 2/10] END ...............C=1000.0, penalty=l1;, score=nan total time=   0.0s
[CV 3/10] END ..............

In [16]:
print(best_model.best_params_)
print('Best Score: ', best_model.best_score_)

{'C': 1000.0, 'penalty': 'l2'}
Best Score:  0.8744987277353691


In [17]:
# Make predictions
grid_predictions = best_model.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":grid_predictions}) 
df_grid.head()

,Actual,Predicted
1903,FALSE POSITIVE,FALSE POSITIVE
1574,CONFIRMED,CONFIRMED
708,CONFIRMED,CONFIRMED
1406,CONFIRMED,CONFIRMED
1062,CANDIDATE,CONFIRMED


In [18]:
# Score the model
best_model.score(X_test_scaled, y_test)

0.8821510297482837

# Save the Model

In [19]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'LR_GradientBoostGridChanged.sav'
joblib.dump(best_model, filename)

['LR_GradientBoostGridChanged.sav']

In [20]:
# Print Classification Report.
from sklearn.metrics import classification_report
print(classification_report(y_test, grid_predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.82      0.69      0.75       422
     CONFIRMED       0.75      0.83      0.79       450
FALSE POSITIVE       0.98      1.00      0.99       876

      accuracy                           0.88      1748
     macro avg       0.85      0.84      0.84      1748
  weighted avg       0.88      0.88      0.88      1748

